In [5]:
class Stack():
    def __init__(self):
        self.inList = None
        self.size = 0
        
    def push(self, v):
        self.size += 1
        self.inList = (v,self.inList)
        
    def pop(self):
        if self.size == 0: assert(0)
        self.size -= 1
        (v,ls) = self.inList
        self.inList = ls
        return v
    
    def __str__(self):
        s = "["
        ls = self.inList
        for _ in range(self.size):
            (v,ls) = ls
            s += str(v)
            if ls!=None: s += ", "
        return s+"]"
        
class ArrayList:
    def __init__(self):
        self.inArray = [0 for i in range(10)]
        self.count = 0
        
    def get(self, i):
        return self.inArray[i]

    def set(self, i, e):
        self.inArray[i] = e

    def length(self):
        return self.count

    def append(self, e):
        self.inArray[self.count] = e
        self.count += 1
        if len(self.inArray) == self.count:
            self._resizeUp()

    def insert(self, i, e):
        for j in range(self.count,i,-1):
            self.inArray[j] = self.inArray[j-1]
        self.inArray[i] = e
        self.count += 1
        if len(self.inArray) == self.count:
            self._resizeUp()
    
    def remove(self, i):
        self.count -= 1
        val = self.inArray[i]
        for j in range(i,self.count):
            self.inArray[j] = self.inArray[j+1]
        return val

    def _resizeUp(self):
        newArray = [0 for i in range(2*len(self.inArray))]
        for j in range(len(self.inArray)):
            newArray[j] = self.inArray[j]
        self.inArray = newArray
        
    def toArray(self):
        return self.inArray[:self.count]

    def __str__(self):
        if self.count == 0: return "[]"
        s = "["
        for i in range(self.count-1): s += str(self.inArray[i])+", "
        return s+str(self.inArray[self.count-1])+"]" 
    
class ArrayListWithUndo(ArrayList):
    def __init__(self):
        # already implemented
        super().__init__()
        self.undos = Stack()

    def set(self, i, v):
        # already implemented
        self.undos.push(("set",i,self.inArray[i]))
        self.inArray[i] = v

    def append(self, v):
        #undo instruction for append is remove
        undo_instruction = ("rem", self.count, None)
        #push undo instruction to stack
        self.undos.push(undo_instruction)
        #append the element
        super().append(v)

    def insert(self, i, v):
        #undo operation for insert is remove
        undo_instruction = ("rem", i, None)
        #push undo instruction to stack
        self.undos.push(undo_instruction)
        #insert the element
        super().insert(i, v)

    def remove(self, i):
        #get the value of the element to be removed
        removed_value = self.get(i)
        #undo instruction for remove is insert
        undo_instruction = ("ins", i, removed_value)
        #push undo instruction to stack
        self.undos.push(undo_instruction)
        #remove the element
        super().remove(i)

    def undo(self):
        #if the stack is empty, return
        if self.undos.size == 0:
            return
        #get the latest undo instruction
        undo_instruction = self.undos.pop()

        op, i, v = undo_instruction
        #if the undo operation is set
        if op == "set":
            super().set(i, v)
        #if the undo operation is remove
        elif op == "rem":
            super().remove(i)
        #if the undo operation is insert
        elif op == "ins":
            super().insert(i, v)

            
    def __str__(self):
        # already implemented
        return str(self.toArray())+"\n-> "+str(self.undos)
    
class NetworkWithUndo:
        def __init__(self, N):
            # already implemented
            self.inArray = ArrayListWithUndo()
            for _ in range(N): self.inArray.append(-1)
            self.undos = Stack()
            self.undos.push(N)
            
            
        def getSize(self):
            # already implemented
            return self.inArray.length()
            
        def add(self):
            #keep track of number of operations
            num_operations = 0
            # Add a new node to the network as its own cluster
            self.inArray.append(-1)
            #since we did one append operation, increment num_operations by 1
            num_operations += 1
            # Push the number of operations onto the undo stack
            self.undos.push(num_operations)
        
        def root(self, i):
        
            #store visited nodes
            visited = []
            #continue iterating until root node met
            while self.inArray.get(i) >= 0:
                visited.append(i)
            
                i = self.inArray.get(i)

                
            
            #for every node visited, make it point to root
            #keep track of number of set operations well do
            num_operations = 0
            for node in visited:
                #make node point to root and increment num_operations
                if i!=self.inArray.get(node):
                    self.inArray.set(node, i)
                    num_operations += 1
            
            #push number of set operations to stack
            self.undos.push(num_operations)
                
            return i

        
        def merge(self, i, j):
            
            #if both nodes are not root nodes, assert 0
            if self.inArray.get(i) >= 0 or self.inArray.get(j) >= 0:
                assert(0)  # One or both are not roots

            # Get sizes of the clusters i and j
            size_i = -self.inArray.get(i)
            size_j = -self.inArray.get(j)

            #keep track of number of operations
            num_operations = 0

            #if i cluster is bigger than j cluster, make root j point to root i
            if size_i > size_j:
                # Make root_j point to root_i
                self.inArray.set(j, i)
                #increment number of operations
                num_operations += 1

                # update size at root i
                self.inArray.set(i, -(size_i + size_j) )
                #increment number of operations
                num_operations += 1
            else:
                # Make root_i point to root_j
                self.inArray.set(i, j)
                #increment number of operations
                num_operations += 1

                # Update size at root j
                self.inArray.set(j, -(size_i + size_j))
                #increment number of operations
                num_operations += 1

            # Push number of operations performed in merge onto the undo stack
            self.undos.push(num_operations)


        
        def undo(self):
            #if empty stack, return
            if self.undos.size == 0:
                return
            
            # Pop the number of operations to undo
            num_operations = self.undos.pop()
            for i in range(num_operations):
                self.inArray.undo()
        
        def toArray(self):
            # already implemented
            return self.inArray.toArray()
                
        def __str__(self):
            # already implemented
            return str(self.toArray())+"\n-> "+str(self.undos)
        
class Gadget:
        def __init__(self):
            # already implemented
            self.inNetwork = NetworkWithUndo(0)
            self.subsize = 0
            self.nameMap = {}
            self.undos = Stack()
            self.helper = {}
            
        def getSize(self):
            # already implemented
            return self.inNetwork.getSize()
            
        def isIn(self, name):
            # already implemented
            return name in self.nameMap
            
        def add(self, name):
            if self.isIn(name):
                self.undos.push(("oth", 0, None))
                return
            # Add the name to the nameMap and track its index
            index = self.inNetwork.getSize()
            self.nameMap[name] = index
            self.inNetwork.add()
            self.subsize += 1

            # Record number of operations so far in helper and push rem operation undos stack
            self.helper[name] = self.undos.size
            self.undos.push(("rem", 1, name))

        def connect(self, name1, name2):
            if not self.isIn(name1) or not self.isIn(name2):
                assert("One or both of the names is not in the nameMap")
            
            index1 = self.nameMap[name1]
            index2 = self.nameMap[name2]

            # Check if already connected
            root1 = self.inNetwork.root(index1)
            root2 = self.inNetwork.root(index2)
            if root1 == root2:
                #if its already connected, push oth with steps=2 to account for the two root operations we did
                self.undos.push(("oth", 2, None))
                return True

            # Merge clusters and update subsize
            self.inNetwork.merge(root1, root2)
            self.subsize -= 1
            #push brk with steps=3 since we merged and did two root operations
            self.undos.push(("brk", 3, None))
            return False

        def clean(self, name):
            if not self.isIn(name):
                assert("Name is not in the nameMap")

            # Get the number of undo steps required
            undo_steps = self.undos.size - self.helper[name]
            self.undo(undo_steps)

        def subnets(self):
            subnet_map = {}
            operations_performed = 0  # Track the number of operations performed on the network

            for name, index in self.nameMap.items():
                root = self.inNetwork.root(index)  # This modifies the internal network state
                
                operations_performed += 1  # Increment for each root call
                if root not in subnet_map:
                    subnet_map[root] = []
                subnet_map[root].append(name)

         
            #push othe with steps number of operations performed to account for number of root operations we did
            self.undos.push(("oth", operations_performed, None))

            return list(subnet_map.values())


        def undo(self, n):
        
            while n > 0 and self.undos.size > 0:
                op, steps, name = self.undos.pop()
                
                if op == "rem":
                    # Remove the name and update subsize
                    del self.nameMap[name]
                    self.subsize -= 1
                elif op == "brk":
                    # Revert a connection and update subsize
                    self.subsize += 1
                
                
                #if the op is oth, we don't need to do anything

                #for the number of steps for each operation, do an undo operation in the network
                for i in range(steps):
                    self.inNetwork.undo()

                n -= 1
        

        
        def toArray(self):
            # already implemented
            A = self.inNetwork.toArray()
            
            for s in self.nameMap:
                i = self.nameMap[s]
                A[i] = (s,A[i])
            return A
            
        def __str__(self):
            # already implemented
            return str(self.toArray())+"\n-> "+str(self.nameMap)+"\n-> "+str(self.undos)
        
    
    